# The Lightning Node Keys Derivation

In later notebooks, we’ll see that a Lightning node uses basepoints and per-commitment points to derive keys throughout the lifecycle of a channel.
This notebook explains how the various basepoints are derived from the node's master seed, and how the per-commitment point is generated.

## Basepoints Derivations

A Lightning node uses a deterministic derivation path based on the [BIP32](https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki) hierarchical deterministic (HD) wallet standard to derive the following basepoints:
    
    * Revocation basepoint
    * Payment basepoint
    * Delayed payment basepoint
    * HTLC basepoint

The [BIP43](https://github.com/bitcoin/bips/blob/master/bip-0043.mediawiki) "Purpose" field defines the first hardened derivation level (m/purpose'/) in hierarchical deterministic (HD) wallets to distinguish between different applications. Lightning Network implementations typically use 1017' as the purpose field in their key derivation paths. Although this is not officially standardized by BIP43, it has become a widely accepted convention among implementations such as LND, Core Lightning (c-lightning), and Eclair. This separation ensures that Lightning keys are derived in isolation from other applications, such as on-chain wallets.

The key derivation path structure used by LND for Lightning keys is:

```
m/1017'/coinType'/keyFamily'/0/index
```

Components of the Path:

* m: The root of the wallet, derived from the master seed.
* 1017': The purpose field indicating that the keys are used for the Lightning Network.
* coin_type': Specifies the blockchain:
    * 0' for Bitcoin mainnet
    * 1' for Bitcoin testnet
* key_family: Specifies the category of key being derived. Common key families include:
    * 1: Revocation basepoints
    * 2: HTLC basepoints
    * 3: Payment basepoints
    * 4: Delayed payment basepoints
* key_index: Specifies the index of the specific key within the family.

## Per Commitment Points

The Per-Commitment Point is a key component of the revocation mechanism in the Lightning Network. It ensures that each commitment transaction is unique and helps prevent cheating during channel closing.
- It is a public key derived from a per-commitment secret.
- Each channel update generates a new per-commitment point.
- The per-commitment point is used to derive the HTLC, delayed and revocation keys.

A node must select an unguessable 256-bit seed for each channel and MUST NOT reveal it. This seed is used to derive all per-commitment secrets for that channel.

Up to (2^48 - 1) per-commitment secrets can be generated using each seed. The first secret used must have index 281474976710655 (0xFFFFFFFFFFFF), and the index is decremented with each subsequent secret.

The I'th secret P must match the output of this algorithm:

```
generate_from_seed(seed, I):
    P = seed
    for B in 47 down to 0:
        if B set in I:
            flip(B) in P
            P = SHA256(P)
    return P
```

### Efficient Storage Strategy
The algorithm described above enables an efficient storage strategy. Instead of storing all per-commitment secrets individually, a node stores secrets only at specific indices corresponding to powers of 2 (e.g., 2^X boundaries). From these stored secrets, all intermediate secrets up to the next boundary can be derived when needed.

Using these "49 pairs" strategy, secrets are stored only at powers-of-2 boundaries. Fox example, if we were to create 1,000 secrets for a specific channel, using this strategy we would store only 11 secrets: 1, 2, 4, 8, 16, 32, 64,128, 256, 512 and 1024.

If each secret is 32 bytes, for 1,000 secrets you have:
- Full storage: 1,000 × 32 = 32,000 bytes.
- Compact storage: 11 × 32 = 352 bytes.

Note: In the example above, we do not use the actual reverse order of indices (as used in practice) to make the explanation easier to follow.


## The Obscured Commitment Number

To help both parties track the commitment number, each commitment transaction encodes its number using the lock time and sequence fields. This encoding is known in the protocol as a state hint. If a party knows the current commitment number, they can use the state hint to quickly identify whether a broadcasted transaction corresponds to a revoked state. To avoid revealing the commitment number directly, an obfuscated state hint is used. This obfuscation is done by XORing the commitment number with 6 random bytes (48 bits) derived deterministically from both participants payment_basepoint. Specifically, the 6 bytes (48 bits) come from the SHA-256 hash of the initiator’s payment_basepoint concatenated with the responder’s payment_basepoint. The obfuscated number is then split across 24 bits of the locktime  and 24 bits of the sequence field, totaling 48 bits for the state hint.

## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    1- The bitcoin core functional test framework directory.
    2- The directory containing Taproot-Lightning-Channels-Workshop.

You'll need to edit these next two lines for your local setup.

In [1]:
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *

import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-09-20T14:51:12.359000Z TestFramework (INFO): PRNG seed is: 8672835170839549742
2025-09-20T14:51:12.361000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_ezdu8whx


In [ ]:
from functions import *
from bip_utils import Bip39SeedGenerator
from hashlib import sha256

# Alice per commitment seed
alice_per_commitment_seed_mnemonic = "auction essay fury nasty crop alien patrol divert express noodle member patch skate average library visit bitter spread oppose leopard cancel install eternal torch"
alice_per_commitment_seed_passphrase = ""
# Generate the seed from the mnemonic
# Generate a 256-bit seed (by truncating the BIP-39 seed)
alice_per_commitment_seed = Bip39SeedGenerator(alice_per_commitment_seed_mnemonic).Generate(alice_per_commitment_seed_passphrase)[:32]
print("Alice Per Commitment Seed", alice_per_commitment_seed.hex())

# Bob per commitment seed
bob_per_commitment_seed_mnemonic = "enroll sponsor smart memory march cover behind undo hockey twist sausage error exercise hollow anger bullet sunset install crime fatal among almost pattern mobile"
bob_per_commitment_seed_passphrase = ""
# Generate the seed from the mnemonic
# Generate a 256-bit seed (by truncating the BIP-39 seed)
bob_per_commitment_seed = Bip39SeedGenerator(bob_per_commitment_seed_mnemonic).Generate(bob_per_commitment_seed_passphrase)[:32]
print("Bob Per Commitment Seed", bob_per_commitment_seed.hex())


# Alice mnemonic and passphrase
alice_node_mnemonic = "walnut meat invite butter addict bargain fault true bundle pyramid biology loop clap blast essence cup crowd throw solution crunch supreme grab hood electric"
alice_node_passphrase = ""

# Generate the seed from the mnemonic
alice_node_seed = Bip39SeedGenerator(alice_node_mnemonic).Generate(alice_node_passphrase)

# Bob mnemonic and passphrase
bob_node_mnemonic = "unique life awesome cinnamon fetch unique yellow squeeze whip chef country foster erupt effort harbor rail tunnel ball ignore right vanish drip stick follow"
bob_node_passphrase = ""

# Generate the seed from the mnemonic
bob_node_seed = Bip39SeedGenerator(bob_node_mnemonic).Generate(bob_node_passphrase)

# The 48-bit commitment number is obscured by XOR with the lower 48 bits of
# SHA256(payment_basepoint from open_channel || payment_basepoint from accept_channel)

# Create payment key derivation object for Alice
alice_payment_key = derivate_key(alice_node_seed, 3)

# Create payment key derivation object for Bob
bob_payment_key = derivate_key(bob_node_seed, 3)

# Concatenate Alice’s and Bob’s payment basepoints
data = alice_payment_key.get_basepoint_compressed() + bob_payment_key.get_basepoint_compressed()
print("Alice Payment BasePoint: ", alice_payment_key.get_basepoint_compressed().hex())
print("Bob Payment BasePoint: ", bob_payment_key.get_basepoint_compressed().hex())

# Compute SHA256 of the concatenated basepoints
to_obscure = sha256(data).digest()

# Convert the SHA256 digest (bytes) to an integer
to_obscure_int = int.from_bytes(to_obscure, "big")

# Extract the lower 48 bits from the SHA256 integer
lower48_to_obscure = to_obscure_int & 0xFFFFFFFFFFFF

print("To obscure commitment number", hex(lower48_to_obscure))

Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
To obscure commitment number 0xd6ff4d17c776
